In [1]:
import pandas as pd
import numpy as np
from operator import itemgetter
import copy
import random
import statistics

In [ ]:
columns_name = []
for i in range(1,11):
    columns_name.append("x"+str(i))
columns_name.append("y")
train_dat = pd.read_excel('hw6_train.xlsx',names=columns_name, header=None) 
test_dat = pd.read_excel('hw6_test.xlsx',names=columns_name, header=None)



def TransformToList(data): #把資料轉成 list 呈現
    data_list = list()
    for i in range(len(data)):
        data_list.append(list(data.iloc[i]))
    return data_list
train_list = TransformToList(train_dat) 
test_list = TransformToList(test_dat)

In [3]:
def get_thresholds(index,dataset): #dataset sent in here should be already sorted 
    thresholds = []
    thresholds.append(dataset[0][index]-1) #first threshold
    for i in range(len(dataset)):
        if(i!=0): #if not the first row
            prev = dataset[i-1][index]
            cur = dataset[i][index]
            if(prev!=cur):
                thresholds.append((prev+cur)/2) #add the mean as one threshold
    thresholds.append(dataset[-1][index]+1) #last threshold
    return thresholds

def data_split(index, value, dataset): #把資料分成左子樹，右子樹
	left,right = list(),list()
	for row in dataset:
		if(row[index] < value):
			left.append(row)
		else:
			right.append(row)
	return left, right

def gini_index(groups): #return the best threshold for that index
    gini = 0.0 #of the threshold
    for group in groups:
        if(len(group)==0): #avoid divided by zero
            continue
        pos_y = 0 #how many positive labels in the group
        for row in group:
            if(row[-1]==1):
                pos_y += 1
        neg_y = len(group) - pos_y
        gini_index = (1 - (pos_y/len(group))**2 - (neg_y/len(group))**2) 
        gini += len(group) * gini_index #to weight different lengths differently
    return gini

def data_same(original_dataset):
    dataset = copy.deepcopy(original_dataset)
    for row in dataset: 
        row.pop() #把label拿掉
    allsame = True
    for i in range(len(dataset)):
        if(dataset[0]!=dataset[i]):
            allsame = False
    return allsame

In [4]:
def get_split(dataset):
    best_index, best_threshold, best_gini, best_groups = 9999, 9999, 9999, None
    terminal = False #判斷是否終止
    for index in range(10):
        sorted_data = sorted(dataset, key=itemgetter(index))
        thresholds = get_thresholds(index,sorted_data)
        for threshold in thresholds:
            groups = data_split(index,threshold,sorted_data)
            gini = gini_index(groups)
            if(gini<best_gini):
                best_gini = gini
                best_index,best_threshold,best_gini,best_groups = index,threshold,gini,groups
    aNode = Node(best_index,best_threshold)

    if(best_gini==0 or data_same(dataset)): #終止條件判斷
        terminal = True
    
    if(terminal):
        aNode.set_label(best_groups)
        return aNode
    else:
        aNode.set_left(get_split(best_groups[0]))
        aNode.set_right(get_split(best_groups[1]))
        return aNode

In [5]:
class Node:
    def __init__(self,index,threshold):
        self.left = None 
        self.right = None
        self.index = index
        self.label_l = 0
        self.label_r = 0
        self.threshold = threshold
    def set_left(self,aNode):
        self.left = aNode
    def set_right(self,aNode):
        self.right = aNode
    def set_label(self,best_groups): 
        if(self.left == None and self.right == None):#沒有子樹的代表是最下方的節點，標示出他們的label
            l_sum,r_sum = 0,0
            
            for row in best_groups[0]: #左子樹
                l_sum += row[-1]
            if(l_sum>0): #如果+1比-1還要多的話，標示為+1(投票機制)
                self.label_l = 1
            else:
                self.label_l = -1
                
            for row in best_groups[1]: #右
                r_sum += row[-1]
            if(r_sum>0): #如果+1比-1還要多的話，標示為+1(投票機制)
                self.label_r = 1
            else:
                self.label_r = -1

In [6]:
def prediction(dataset,aNode):
    index, threshold = aNode.index, aNode.threshold
    #先依index排序，再split成兩組
    sorted_data = sorted(dataset, key=itemgetter(index))
    groups = data_split(index,threshold,dataset)
    err = 0
    results = []
    if(aNode.left == None): # it doesn't have any child node, we should do the classification
        #左節點
        for row in groups[0]:
            results.append((row,aNode.label_l))
            if(row[-1]!=aNode.label_l):
                err += 1
        #右節點
        for row in groups[1]:
            results.append((row,aNode.label_r))
            if(row[-1]!=aNode.label_r):
                err += 1
    
        return err,results

    else: # the node has its child nodes, keep traversing
        err_l,results_l = prediction(groups[0],aNode.left)
        err_r,results_r = prediction(groups[1],aNode.right)
        err += err_l+err_r #add the classification error of two child nodes to err

        for element in results_l:
            results.append(element)
        for element in results_r:
            results.append(element)
        return err,results


In [7]:
def Bagging(fraction,dataset):
    bagging_len = int(fraction*len(dataset))
    bagging_data = []
    index_list = random.choices(range(0,len(dataset)),k=bagging_len)
    for index in index_list:
      bagging_data.append(dataset[index])
    return bagging_data

def RandomForest(training_data,testing_data,NumberofTrees): #隨機張森林
    errs = []
    results = {}
    for i in range(NumberofTrees): #the bagging process
        temp_train = Bagging(0.5,training_data)
        aNode = get_split(temp_train)
        err,result = prediction(testing_data,aNode)
        errs.append(err)
        for row in result:
            if(tuple(row[0]) not in results):
                results[tuple(row[0])] = row[1]
            else:
                results[tuple(row[0])] += row[1]
    return statistics.mean(errs)/len(testing_data), results

In [8]:
def sign(x):
    if(x>0):
        return 1
    else:
        return -1
def ErrorFunction(results):
    error = 0
    for key in results:
        if(sign(results[key])!=key[-1]):
            error += 1
    return error/len(results)

In [7]:
# ein of CART
aNode = get_split(train_list)
err,result = prediction(train_list,aNode)
print(err/len(train_list))

0.0


In [8]:
# eout of CART
aNode = get_split(train_list)
err,result = prediction(test_list,aNode)
print(err/len(train_list))

0.166


In [14]:
# ein of Random Forest
err,results = RandomForest(train_list,train_list,200)
print(ErrorFunction(results))

0.013


In [17]:
# # eout of Random Forest
err,results = RandomForest(train_list,test_list,200)
print(ErrorFunction(results))

0.152


In [9]:
def RandomForest_oob(training_data,NumberofTrees): 
    errs = []
    results = {}
    for i in range(NumberofTrees): #the bagging process
        oob_data = []
        temp_train = Bagging(0.5,training_data)
        aNode = get_split(temp_train)
        #find oob data
        for row in training_data:
          if(row not in temp_train):
            oob_data.append(row)
        err,result = prediction(oob_data,aNode)
        errs.append(err)
        for row in result:
            if(tuple(row[0]) not in results):
                results[tuple(row[0])] = row[1]
            else:
                results[tuple(row[0])] += row[1]
    return statistics.mean(errs)/len(training_data),results

In [11]:
# eoob of Random Forest
err_oob,results_oob = RandomForest_oob(train_list,200)
ErrorFunction(results_oob)

0.069